In [2]:
# https://docs.google.com/spreadsheets/d/1y3rZuCJkkNQoulXL9OzkvFfIPuKZpl2c7dTsGRoyE2g/edit?gid=88364224#gid=88364224

In [3]:
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
import pyodbc
import pandas as pd
import numpy as np
from datetime import datetime 

import warnings
warnings.filterwarnings("ignore")


In [4]:
#Google Sheets
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("python-459011-73acbdff4227.json", scope)
client = gspread.authorize(creds)

In [5]:
master_sheet = client.open_by_key("1y3rZuCJkkNQoulXL9OzkvFfIPuKZpl2c7dTsGRoyE2g").worksheet("Summary")
index = get_as_dataframe(master_sheet, evaluate_formulas=True, header=1)
index.dropna(how='all', inplace=True)
index.reset_index(drop=True, inplace=True)

In [6]:
# index.head()

In [7]:
# Filter only active projects
active_projects = index[index["Active Flag"] == 1]

In [8]:
# === SQL Server connection details ===
server = '157.119.230.120,4071'
database = 'PowerBI_LISV2'
username = 'sa'
password = 'Y0m@Sql.!.123'

conn_str = f"""
    DRIVER={{ODBC Driver 17 for SQL Server}};
    SERVER={server};
    DATABASE={database};
    UID={username};
    PWD={password};
    TrustServerCertificate=yes;
"""


In [9]:
all_frames = []  # collect per-project DataFrames here

for idx, row in active_projects.iterrows():
    project_name = row["Project Name"]
    schema = row["SQL Schema Name"].strip("[]")  # e.g., [dbo] -> dbo

    # print(f"\n🚀 Processing Project: {project_name} | Schema: {schema}")
    try:
        with pyodbc.connect(conn_str) as conn:
            query = f"""
            WITH t AS (
                SELECT 
                    LISStoreCode,
                    PERCENTILE_CONT(0.5) WITHIN GROUP
                        (ORDER BY TRY_CONVERT(float, InLatitude))
                        OVER (PARTITION BY LISStoreCode) AS MedianLatitude,
                    PERCENTILE_CONT(0.5) WITHIN GROUP
                        (ORDER BY TRY_CONVERT(float, InLongitude))
                        OVER (PARTITION BY LISStoreCode) AS MedianLongitude
                FROM [{database}].[{schema}].[ISPStoreAttendance_NormalExport]
                WHERE [LISStoreCode] NOT LIKE '%SP'
                  AND LOWER([LISStoreCode]) NOT LIKE '%test%'
                  AND TRY_CONVERT(float, InLatitude)  IS NOT NULL
                  AND TRY_CONVERT(float, InLongitude) IS NOT NULL
            )
            SELECT DISTINCT LISStoreCode, MedianLatitude, MedianLongitude
            FROM t;
            """
            df = pd.read_sql(query, conn)

        df["SchemaName"] = schema  # optional: keep track of source
        all_frames.append(df)
        print(f"✔️ {schema}: {len(df)} rows")
    except Exception as e:
        print(f"❌ Error processing {project_name} ({schema}): {e}")

# Combine after the loop
if all_frames:
    final_df = pd.concat(all_frames, ignore_index=True)
    print("\nPreview:")
    print(final_df.head())
else:
    print("No data gathered.")


✔️ AdaniWilmerManagement: 5351 rows
✔️ CholayilManagement: 2177 rows
✔️ DaburPharmaManagement: 16372 rows
✔️ DanoneGTManagement: 14505 rows
✔️ DelMonteManagement: 767 rows
✔️ GCPL_SAMTManagement: 1041 rows
✔️ GCPLFoodsManagement: 1030 rows
✔️ HaleonManagement: 5443 rows
✔️ HimalayaTurmericGTManagement: 170 rows
✔️ Jayanti_GroupManagement: 335 rows
✔️ JyothyLabsManagement: 703 rows
✔️ CamlinManagement: 594 rows
✔️ KRBLGTManagement: 1510 rows
✔️ MarsManagement: 10065 rows
✔️ McnroeManagement: 1515 rows
✔️ PatanjaliManagement: 591 rows
✔️ PidiliteManagement: 2680 rows
✔️ PiramalManagement: 207 rows
✔️ BeamSuntoryManagement: 10812 rows
✔️ ZydusGTManagement: 6581 rows

Preview:
   LISStoreCode  MedianLatitude  MedianLongitude             SchemaName
0  AD-Store-005       22.719771        75.907401  AdaniWilmerManagement
1  AD-Store-014       22.717900        75.908289  AdaniWilmerManagement
2  AD-Store-015       22.741379        75.878514  AdaniWilmerManagement
3  AD-Store-016       22.73512

In [10]:
all_frames2 = []  # collect per-project DataFrames here

for idx, row in active_projects.iterrows():
    project_name = row["Project Name"]
    schema = row["SQL Schema Name"].strip("[]")  # e.g., [dbo] -> dbo

    # print(f"\n🚀 Processing Project: {project_name} | Schema: {schema}")
    try:
        with pyodbc.connect(conn_str) as conn:
            query = f"""
            SELECT * FROM [{database}].[{schema}].StoreMaster WHERE LOWER([LISStoreCode]) NOT LIKE '%test%';
            """
            df = pd.read_sql(query, conn)

        df["SchemaName"] = schema  # optional: keep track of source
        all_frames2.append(df)
        print(f"✔️ {schema}: {len(df)} rows")
    except Exception as e:
        print(f"❌ Error processing {project_name} ({schema}): {e}")

# Combine after the loop
if all_frames:
    StoreMaster_df = pd.concat(all_frames2, ignore_index=True)
    # print("\nPreview:")
    # print(StoreMaster_df.head())
else:
    print("No data gathered.")


✔️ AdaniWilmerManagement: 5710 rows
✔️ CholayilManagement: 3474 rows
✔️ DaburPharmaManagement: 13517 rows
✔️ DanoneGTManagement: 14851 rows
✔️ DelMonteManagement: 85680 rows
✔️ GCPL_SAMTManagement: 1163 rows
✔️ GCPLFoodsManagement: 144501 rows
✔️ HaleonManagement: 7108 rows
✔️ HimalayaTurmericGTManagement: 212 rows
✔️ Jayanti_GroupManagement: 637 rows
✔️ JyothyLabsManagement: 1079 rows
✔️ CamlinManagement: 680 rows
✔️ KRBLGTManagement: 3807 rows
✔️ MarsManagement: 11099 rows
✔️ McnroeManagement: 2304 rows
✔️ PatanjaliManagement: 828 rows
✔️ PidiliteManagement: 3128 rows
✔️ PiramalManagement: 327 rows
✔️ BeamSuntoryManagement: 15580 rows
✔️ ZydusGTManagement: 11034 rows


In [11]:
final_storemaster= pd.merge(StoreMaster_df,final_df,on = ['LISStoreCode','SchemaName'], how = 'left')

In [12]:
final_storemaster.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 326719 entries, 0 to 326718
Data columns (total 34 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   StoreMasterId    326719 non-null  int64  
 1   ISPStoreId       326719 non-null  int64  
 2   ClientStoreCode  191865 non-null  object 
 3   LISStoreCode     326719 non-null  object 
 4   StoreName        326719 non-null  object 
 5   DistributorCode  37197 non-null   object 
 6   DistributorName  37851 non-null   object 
 7   KYCStatus        141 non-null     object 
 8   Division         7100 non-null    object 
 9   ChainCode        326719 non-null  object 
 10  SubChainName     34007 non-null   object 
 11  StoreType        57803 non-null   object 
 12  Format           326719 non-null  object 
 13  StoreClub        138 non-null     object 
 14  StoreClass       10947 non-null   object 
 15  StoreAddress     235660 non-null  object 
 16  StoreStatus      57616 non-null   obje

In [13]:
# final_storemaster.head()

In [14]:
# final_storemaster[final_storemaster['LISStoreCode']=='AD-Store-7508']

In [15]:
# final_storemasterx = final_storemaster[final_storemaster['SchemaName_x']=='AdaniWilmerManagement']

In [16]:
final_storemaster.to_csv('final_storemaster.csv')

In [17]:
# final_df.to_csv('final_df_slatlong.csv')

In [18]:
# StoreMaster_df.to_csv('StoreMaster_df.csv')